In [1]:
import json
import csv
#import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
#import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Quora/Data/VITQuora.csv')


In [3]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers
0,0,Vellore-Institute-of-Technology-Vellore,Can I get into the ECE branch at VIT Chennai w...,0,1,0/1,No,Requested,2020-07-26,https://www.quora.com/Can-I-get-into-the-ECE-b...,Not sure. I do not knowcwhat the JEE is but i ...
1,1,Vellore-Institute-of-Technology-Vellore,My son got 89.6% overall in class 12 and 87.8 ...,2,2,2/2,No,Followed,2020-07-26,https://www.quora.com/My-son-got-89-6-overall-...,You can other streams in Thapar university but...
2,2,Vellore-Institute-of-Technology-Vellore,I got 89.4% in CBSE Boards and 94.43 percentil...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/I-got-89-4-in-CBSE-Board...,First of all congratulations to you that you h...
3,3,Vellore-Institute-of-Technology-Vellore,Can I get into VIT Vellore with the CSE stream...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/Can-I-get-into-VIT-Vello...,There is less chances of getting CSE or IT str...
4,4,Vellore-Institute-of-Technology-Vellore,I got 92 percent in 12th class. Can I get into...,1,2,1/2,No,Followed,2020-07-26,https://www.quora.com/I-got-92-percent-in-12th...,"If you just have the boards percentage, then i..."
...,...,...,...,...,...,...,...,...,...,...,...
1568,1568,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,Choose wisely: Institution reputation: BITS ha...
1569,1569,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,"Take vit Vellore, it's one of the the most rep..."
1570,1570,Vellore-Institute-of-Technology-Vellore,Can I get an AC room in 2nd year with 6 GPA in...,2,1,2/1,No,Followed,2020-02-22,https://www.quora.com/Can-I-get-an-AC-room-in-...,"Yes, it might be possible not with the regular..."
1571,1571,Vellore-Institute-of-Technology-Vellore,Should I drop one more year for the GATE or jo...,3,1,3/1,No,Followed,2020-02-22,https://www.quora.com/Should-I-drop-one-more-y...,I don't know the placement record of VIT. So I...


In [4]:
df=df.reset_index()


In [5]:
df=df.drop_duplicates()

In [6]:
df=df.drop(['index'],axis=1)

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['answers']]

In [8]:
#classifying sentiments as positive,negative and neutral
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

df['Sentiment'] = df['compound'].apply(lambda x: sentiment(x))

In [9]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment
0,0,Vellore-Institute-of-Technology-Vellore,Can I get into the ECE branch at VIT Chennai w...,0,1,0/1,No,Requested,2020-07-26,https://www.quora.com/Can-I-get-into-the-ECE-b...,Not sure. I do not knowcwhat the JEE is but i ...,0.9576,Positive
1,1,Vellore-Institute-of-Technology-Vellore,My son got 89.6% overall in class 12 and 87.8 ...,2,2,2/2,No,Followed,2020-07-26,https://www.quora.com/My-son-got-89-6-overall-...,You can other streams in Thapar university but...,0.9803,Positive
2,2,Vellore-Institute-of-Technology-Vellore,I got 89.4% in CBSE Boards and 94.43 percentil...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/I-got-89-4-in-CBSE-Board...,First of all congratulations to you that you h...,0.9590,Positive
3,3,Vellore-Institute-of-Technology-Vellore,Can I get into VIT Vellore with the CSE stream...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/Can-I-get-into-VIT-Vello...,There is less chances of getting CSE or IT str...,0.7102,Positive
4,4,Vellore-Institute-of-Technology-Vellore,I got 92 percent in 12th class. Can I get into...,1,2,1/2,No,Followed,2020-07-26,https://www.quora.com/I-got-92-percent-in-12th...,"If you just have the boards percentage, then i...",-0.6110,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,1568,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,Choose wisely: Institution reputation: BITS ha...,0.9832,Positive
1569,1569,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,"Take vit Vellore, it's one of the the most rep...",0.9517,Positive
1570,1570,Vellore-Institute-of-Technology-Vellore,Can I get an AC room in 2nd year with 6 GPA in...,2,1,2/1,No,Followed,2020-02-22,https://www.quora.com/Can-I-get-an-AC-room-in-...,"Yes, it might be possible not with the regular...",0.9584,Positive
1571,1571,Vellore-Institute-of-Technology-Vellore,Should I drop one more year for the GATE or jo...,3,1,3/1,No,Followed,2020-02-22,https://www.quora.com/Should-I-drop-one-more-y...,I don't know the placement record of VIT. So I...,0.9898,Positive


In [10]:
df['score'] = pd.cut(df['compound'], bins=5, labels=[1, 2, 3, 4, 5])
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score
0,0,Vellore-Institute-of-Technology-Vellore,Can I get into the ECE branch at VIT Chennai w...,0,1,0/1,No,Requested,2020-07-26,https://www.quora.com/Can-I-get-into-the-ECE-b...,Not sure. I do not knowcwhat the JEE is but i ...,0.9576,Positive,5
1,1,Vellore-Institute-of-Technology-Vellore,My son got 89.6% overall in class 12 and 87.8 ...,2,2,2/2,No,Followed,2020-07-26,https://www.quora.com/My-son-got-89-6-overall-...,You can other streams in Thapar university but...,0.9803,Positive,5
2,2,Vellore-Institute-of-Technology-Vellore,I got 89.4% in CBSE Boards and 94.43 percentil...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/I-got-89-4-in-CBSE-Board...,First of all congratulations to you that you h...,0.9590,Positive,5
3,3,Vellore-Institute-of-Technology-Vellore,Can I get into VIT Vellore with the CSE stream...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/Can-I-get-into-VIT-Vello...,There is less chances of getting CSE or IT str...,0.7102,Positive,5
4,4,Vellore-Institute-of-Technology-Vellore,I got 92 percent in 12th class. Can I get into...,1,2,1/2,No,Followed,2020-07-26,https://www.quora.com/I-got-92-percent-in-12th...,"If you just have the boards percentage, then i...",-0.6110,Negative,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,1568,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,Choose wisely: Institution reputation: BITS ha...,0.9832,Positive,5
1569,1569,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,"Take vit Vellore, it's one of the the most rep...",0.9517,Positive,5
1570,1570,Vellore-Institute-of-Technology-Vellore,Can I get an AC room in 2nd year with 6 GPA in...,2,1,2/1,No,Followed,2020-02-22,https://www.quora.com/Can-I-get-an-AC-room-in-...,"Yes, it might be possible not with the regular...",0.9584,Positive,5
1571,1571,Vellore-Institute-of-Technology-Vellore,Should I drop one more year for the GATE or jo...,3,1,3/1,No,Followed,2020-02-22,https://www.quora.com/Should-I-drop-one-more-y...,I don't know the placement record of VIT. So I...,0.9898,Positive,5


In [11]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [12]:
df['followers_n']=normalize('Followers')

In [13]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n
0,0,Vellore-Institute-of-Technology-Vellore,Can I get into the ECE branch at VIT Chennai w...,0,1,0/1,No,Requested,2020-07-26,https://www.quora.com/Can-I-get-into-the-ECE-b...,Not sure. I do not knowcwhat the JEE is but i ...,0.9576,Positive,5,0.000000
1,1,Vellore-Institute-of-Technology-Vellore,My son got 89.6% overall in class 12 and 87.8 ...,2,2,2/2,No,Followed,2020-07-26,https://www.quora.com/My-son-got-89-6-overall-...,You can other streams in Thapar university but...,0.9803,Positive,5,0.076923
2,2,Vellore-Institute-of-Technology-Vellore,I got 89.4% in CBSE Boards and 94.43 percentil...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/I-got-89-4-in-CBSE-Board...,First of all congratulations to you that you h...,0.9590,Positive,5,0.038462
3,3,Vellore-Institute-of-Technology-Vellore,Can I get into VIT Vellore with the CSE stream...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/Can-I-get-into-VIT-Vello...,There is less chances of getting CSE or IT str...,0.7102,Positive,5,0.038462
4,4,Vellore-Institute-of-Technology-Vellore,I got 92 percent in 12th class. Can I get into...,1,2,1/2,No,Followed,2020-07-26,https://www.quora.com/I-got-92-percent-in-12th...,"If you just have the boards percentage, then i...",-0.6110,Negative,1,0.038462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,1568,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,Choose wisely: Institution reputation: BITS ha...,0.9832,Positive,5,0.153846
1569,1569,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,"Take vit Vellore, it's one of the the most rep...",0.9517,Positive,5,0.153846
1570,1570,Vellore-Institute-of-Technology-Vellore,Can I get an AC room in 2nd year with 6 GPA in...,2,1,2/1,No,Followed,2020-02-22,https://www.quora.com/Can-I-get-an-AC-room-in-...,"Yes, it might be possible not with the regular...",0.9584,Positive,5,0.076923
1571,1571,Vellore-Institute-of-Technology-Vellore,Should I drop one more year for the GATE or jo...,3,1,3/1,No,Followed,2020-02-22,https://www.quora.com/Should-I-drop-one-more-y...,I don't know the placement record of VIT. So I...,0.9898,Positive,5,0.115385


In [14]:
engagement_median = df['followers_n'].median()
df["engagement"] = df['followers_n'].apply(lambda x: 1 if x >= engagement_median else 0)

In [15]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [16]:
df['sentiment_score']=df['score']
df_eng1=df[df.engagement==1]
#if engagement score is greater than the median, increase the positive and negative sentiment
df_eng1['sentiment_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df[df.engagement==1]=df_eng1
df

C:\Users\Kirthika\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n,engagement,sentiment_score
0,0,Vellore-Institute-of-Technology-Vellore,Can I get into the ECE branch at VIT Chennai w...,0,1,0/1,No,Requested,2020-07-26,https://www.quora.com/Can-I-get-into-the-ECE-b...,Not sure. I do not knowcwhat the JEE is but i ...,0.9576,Positive,5,0.000000,0,5
1,1,Vellore-Institute-of-Technology-Vellore,My son got 89.6% overall in class 12 and 87.8 ...,2,2,2/2,No,Followed,2020-07-26,https://www.quora.com/My-son-got-89-6-overall-...,You can other streams in Thapar university but...,0.9803,Positive,5,0.076923,1,5
2,2,Vellore-Institute-of-Technology-Vellore,I got 89.4% in CBSE Boards and 94.43 percentil...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/I-got-89-4-in-CBSE-Board...,First of all congratulations to you that you h...,0.9590,Positive,5,0.038462,0,5
3,3,Vellore-Institute-of-Technology-Vellore,Can I get into VIT Vellore with the CSE stream...,1,1,1/1,No,Followed,2020-07-26,https://www.quora.com/Can-I-get-into-VIT-Vello...,There is less chances of getting CSE or IT str...,0.7102,Positive,5,0.038462,0,5
4,4,Vellore-Institute-of-Technology-Vellore,I got 92 percent in 12th class. Can I get into...,1,2,1/2,No,Followed,2020-07-26,https://www.quora.com/I-got-92-percent-in-12th...,"If you just have the boards percentage, then i...",-0.6110,Negative,1,0.038462,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,1568,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,Choose wisely: Institution reputation: BITS ha...,0.9832,Positive,5,0.153846,1,5
1569,1569,Vellore-Institute-of-Technology-Vellore,I got enough rank to get CS in VIT Vellore and...,4,3,4/3,No,Followed,2020-02-26,https://www.quora.com/I-got-enough-rank-to-get...,"Take vit Vellore, it's one of the the most rep...",0.9517,Positive,5,0.153846,1,5
1570,1570,Vellore-Institute-of-Technology-Vellore,Can I get an AC room in 2nd year with 6 GPA in...,2,1,2/1,No,Followed,2020-02-22,https://www.quora.com/Can-I-get-an-AC-room-in-...,"Yes, it might be possible not with the regular...",0.9584,Positive,5,0.076923,1,5
1571,1571,Vellore-Institute-of-Technology-Vellore,Should I drop one more year for the GATE or jo...,3,1,3/1,No,Followed,2020-02-22,https://www.quora.com/Should-I-drop-one-more-y...,I don't know the placement record of VIT. So I...,0.9898,Positive,5,0.115385,1,5


In [18]:
df.rename(columns = {'score':'old_score'}, inplace = True) 

In [19]:
df.to_csv('../../Datasets/Quora/WSA/VITQuoraWSA.csv')